<a href="https://colab.research.google.com/github/YChen1212/SystemAccount_Risk_Analysis/blob/main/%E5%B8%B3%E8%99%9F%E6%AC%8A%E9%99%90%E9%83%A8%E9%96%80%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **建立帳號權限部門表**

In [2]:
# 安裝faker套件，以生成假資料
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.2 MB/s eta 0:00:00


In [3]:
# 匯入套件
import pandas as pd
import numpy as np
import random
from google.colab import files
from faker import Faker

In [4]:
# 使用Faker函式庫建立一個以臺灣（zh_TW）之假資料生成器
fake = Faker('zh_TW')
# 設定隨機種子為50，以確保隨機結果重現
random.seed(50)

In [5]:
# 設定使用者數量
total_user=50
# 設定各部門與職位對應的權限，字典格式為{'部門':{'職稱':['權限']}}
ia_dept_job = {'IA':{'主任稽核':['查閱'], '稽核人員':['查閱']}}
dept_job = {
    'HR':{'人資主管':['薪資管理'], '人資':['招聘作業','員工資料維護']},
    'FIN':{'財務主管':['傳票核准'], '會計':['編製會計憑證','費用報支審核'], '出納':['付款','出納確認'] },
    'ITS':{'資訊主管':['SUPER_USER'], '資訊人員':['SUPER_USER']},
    'SALES':{'銷售主管':['價格審核', '銷售報表查詢'], '銷售人員':['銷售訂單維護','客戶資料維護']},
    'PD':{'採購主管':['採購審核'],'採購人員':['採購訂單輸入','產品定價管理']},
    'R&D':{'研發主管':['研發文件審核'], '研發人員':['研發專案資料維護']},
    }

In [6]:
# 建立空白list，以讓資料填入
account_data = []

# 設定到職日為code撰寫當下往回推1年~5年(2020/8~2024/8)，並使用fake.date_between隨機產出區間內之日期，為讓後續欄位account_estd=hire_date，故獨立設定變數
hire_date = fake.date_between(start_date='-5y', end_date='-1y')

# 預設資料表必定包含一主任稽核及一稽核人員，將該兩列資料固定加入list
account_data.append({
    "user_id": "E0001",
    "user_name": "user0001",
    "user_account": "ERP0001",
    "department": "IA",
    "job_title": "主任稽核",
    "permissions":ia_dept_job["IA"]["主任稽核"],
    'is_active':True,
    'status':'在職',
    'last_logon_date':fake.date_between(start_date='-30d', end_date='-10d'), # 設立帳號最後登入日為code撰寫當下往回推10天~30天(2025/7~2025/8)，並使用fake.date_between隨機產出區間內之日期
    'hire_date':hire_date,
    'account_estd': hire_date,
    'terminate_date':''
})
hire_date = fake.date_between(start_date='-5y', end_date='-1y')
account_data.append({
    "user_id": "E0002",
    "user_name": "user0002",
    "user_account": "ERP0002",
    "department": "IA",
    "job_title": "稽核人員",
    "permissions":ia_dept_job["IA"]["稽核人員"],
    'is_active':True,
    'status':'在職',
    'last_logon_date':fake.date_between(start_date='-30d', end_date='-10d'),
    'hire_date':hire_date,
    'account_estd': hire_date,
    'terminate_date':''
})



In [7]:
# 隨機生成其餘使用者的帳號資料，涵蓋不同部門和職位
for i in range(3, total_user+1):
  user_id = f"E{i:04d}"
  user_name = f"user{i:04d}"
  user_account = f"ERP{i:04d}"
  last_logon_date = fake.date_between(start_date  = '-30d', end_date = '-10d')
  hire_date = fake.date_between(start_date  = '-5y', end_date = '-1y')
  account_estd=hire_date
  is_active = np.random.choice([True, False], p=[0.8, 0.2]) # 隨機設定是否在職
  if is_active == True: # 根據是否在職決定狀態
    status = '在職'
  else:
    status = '離職'
  if status == '離職':
    terminate_date=last_logon_date # 預設若人員離職其帳號登入日為離職日當日
  else:
    terminate_date=''
  dept = random.choice(list(dept_job.keys())) # 隨機選擇部門
  job = random.choice(list(dept_job[dept].keys())) # 根據部門，隨機選擇職位
  permissions = dept_job[dept][job] # 根據部門和職位，隨機選擇權限
  account_data.append(
      {
          "user_id":user_id,
          "user_name":user_name,
          'user_account':user_account,
          'account_estd':account_estd,
          'is_active':is_active,
          'last_logon_date':last_logon_date,
          'hire_date':hire_date, 'status':status,
          'terminate_date':terminate_date,
          'department':dept,
          'job_title':job,
          'permissions':permissions
       }
      )

In [9]:
# 將account_data轉換為DataFrame
df = pd.DataFrame(account_data)

In [10]:
# 將Dataframe(df)轉換為csv格式並儲存為ALL.csv，不包含index
df_account = df.to_csv("ALL.csv", index=False)
# 下載ALL.csv至本地
files.download("ALL.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>